In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from multirtd.dubins_model import dubins_traj, dubins_traj_new, linearize_dynamics

%load_ext autoreload
%autoreload 2

### Monte carlo

In [ ]:
# Initial state (no uncertainty)
x0 = np.array([0, 0, 0])  # [x, y, theta]

# Monte carlo over 1000 samples
dt = 0.1
N_MC = 1000
K = 100  # Number of timesteps
sigma = 0.03  # rotational velocity sigma

u = np.zeros((K, 2))
u[:, 0] = 1  # constant velocity

x_samples = []
for i in range(N_MC):
    u[:, 1] = np.random.normal(0, sigma, K)  # random rotational velocity
    x = dubins_traj(x0, u, dt)
    x_samples.append(x)

# Plot the trajectory
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
for x in x_samples:
    ax1.plot(x[:,0], x[:,1], 'b', alpha=0.1)
    ax2.scatter(x[-1,0], x[-1,1])
plt.axis('equal')
plt.show()

In [ ]:
dt = 1.0
K = 30
sigma = 0.3

from multirtd.dubins_model import linearize_dynamics
# # Linearized dynamics
# def linearize_dynamics(x, u, dt):
#     A = np.eye(3)
#     A[0, 2] = -u[0] * dt * np.sin(x[2])
#     A[1, 2] = u[0] * dt * np.cos(x[2])
#     B = np.zeros((3, 2))
#     B[0, 0] = dt * np.cos(x[2])
#     B[1, 0] = dt * np.sin(x[2])
#     B[2, 1] = dt
#     return A, B

u = np.zeros((K, 2))
u[:, 0] = 1  # constant velocity
#u[:, 1] = np.random.normal(0, sigma, K)  # random rotational velocity
u[:, 1] = 0.1
x = dubins_traj_new(x0, u, dt)

# Compare linearized dynamics with actual dynamics
x_lin = np.zeros((K, 3))
x_lin[0] = x0
for i in range(K-1):
    A, B = linearize_dynamics(x[i], u[i], dt)
    x_lin[i+1] = A @ x_lin[i] + B @ u[i]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.axis('equal')
ax1.plot(x[:,0], x[:,1], 'b', label='Actual')
ax1.plot(x_lin[:,0], x_lin[:,1], 'r', label='Linearized')

In [ ]:
dt = 2.0
K = 30
u = np.zeros((K, 2))
u[:, 0] = 1 
u[:, 1] = 0.01

# Compare dubins_traj and dubins_traj_new
x = dubins_traj(x0, u, dt)
x_new = dubins_traj_new(x0, u, dt)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.plot(x[:,0], x[:,1], 'b')
ax1.plot(x_new[:,0], x_new[:,1], 'r')

### Akshay's reachability

### RRBT Propagation

Relies on nominal trajectory - linearizes dubin's dynamics to get $(A_t, B_t)$ around nominal trajectory

Uses ranging beacon measurement setup, linearized measurement model around nominal trajectory to get $(C_t)$

Uses LQR to get $(K_t)$

In [ ]:
from multirtd.dubins_reachability import generate_robot_matrices

In [ ]:
def system_matrices(x, u, Q_lqr, R_lqr, dt):
    A, B = linearize_dynamics(x, u, dt)
    Q = Q_lqr
    R = R_lqr
    return A, B, Q, R

In [ ]:
dt = 0.1
N = 30
x0 = np.array([0, 0, 0])
u_nom = np.zeros((N, 2))
u_nom[:, 0] = 1
u_nom[:, 1] = 0.1
x_nom = dubins_traj(x0, u_nom, dt)



### Robotics toolbox

In [ ]:
from roboticstoolbox.mobile import Bicycle, RandomPath

V = np.diag([0.2, np.radians(0.5)])
robot = Bicycle(covar=V, dt=0.1)
robot.control = RandomPath(workspace=10)

In [ ]:
N_MC = 100
x_samples = []
fig, ax = plt.subplots(figsize=(5, 5))
for i in range(N_MC):
    x = robot.run(10)
    x_samples.append(x)
    ax.plot(x[:, 0], x[:, 1])
ax.axis('equal')

### Closed loop simulation

In [ ]:
def diff_drive(vl, vr, sigma, N, d, dt):
    """Diff drive model with uncertainty in the wheel velocities.

    d : distance between wheels

    """
    x = np.zeros((N,3))
    for i in range(N-1):
        noise = np.random.normal(0, sigma, 2)
        dx = np.array([0.5 * (vl + vr + noise[0] + noise[1]) * np.cos(x[i,2]),
                       0.5 * (vl + vr + noise[0] + noise[1]) * np.sin(x[i,2]),
                       (vr - vl + noise[0] - noise[1]) / d])
        x[i+1] = x[i] + dx * dt
    return x

In [ ]:
N_MC = 100
N = 100
d = 0.1
dt = 0.1
sigma = 0.01
vl = 0.15
vr = 0.2

x_MC = np.zeros((N_MC, N, 3))
for i in range(N_MC):
    x = diff_drive(vl, vr, sigma, N, d, dt)
    x_MC[i] = x
    plt.plot(x[:,0], x[:,1])
plt.axis('equal')
plt.show()

In [ ]:
for i in range(N_MC):
    plt.plot(x_MC[i,0:N:10,0], x_MC[i,0:N:10,1], 'b.', markersize=1)
    #plt.plot(x_MC[i,-1,0], x_MC[i,-1,1], 'b.', markersize=1)
plt.axis('equal')
plt.show()

In [ ]:
# Nominal trajectory
v = (vl + vr) / 2
w = (vr - vl) / d
u_nom = np.array([[v, w]] * N)
x_nom = dubins_traj(x0, u_nom, dt)

plt.plot(x_nom[:, 0], x_nom[:, 1])
plt.axis('equal')
plt.show()

In [ ]:
def diff_drive_var_sigma(vl, vr, sigmas, N, d, dt):
    """Diff drive model with uncertainty in the wheel velocities.

    d : distance between wheels

    """
    x = np.zeros((N,3))
    for i in range(N-1):
        noise = np.random.normal(0, sigmas[i], 2)
        dx = np.array([0.5 * (vl + vr + noise[0] + noise[1]) * np.cos(x[i,2]),
                       0.5 * (vl + vr + noise[0] + noise[1]) * np.sin(x[i,2]),
                       (vr - vl + noise[0] - noise[1]) / d])
        x[i+1] = x[i] + dx * dt
    return x

In [ ]:
N_MC = 100
N = 100
d = 0.1
dt = 0.1
sigmas = np.zeros(N)
sigmas[:50] = 0.01
sigmas[50:] = 0.001
vl = 0.19
vr = 0.2

x_MC = np.zeros((N_MC, N, 3))
for i in range(N_MC):
    x = diff_drive_var_sigma(vl, vr, sigmas, N, d, dt)
    x_MC[i] = x
    plt.plot(x[:,0], x[:,1])
plt.axis('equal')
plt.show()

In [ ]:
# Error state linearization

x_err = np.zeros((N, 3))

for i in range(N):
    A, B = linearize_dynamics(x_nom[i], u_nom[i], dt)

In [ ]:
from matplotlib.patches import Ellipse

def plot_ellipse(ax, c, Sigma):
    """Plot 0.95 confidence ellipse from center and covariance matrix
    
    """
    vals, vecs = np.linalg.eig(Sigma)
    order = vals.argsort()[::-1]
    vals = vals[order]
    vecs = vecs[:, order]
    theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))
    width, height = 2 * np.sqrt(5.991) * np.sqrt(vals)
    ellip = Ellipse(xy=c, width=width, height=height, angle=theta, alpha=0.5)
    ax.add_artist(ellip)


In [ ]:
# RRBT

Sigma = np.zeros((3, 3))
Q = sigma**2 * np.eye(2)

fig, ax = plt.subplots(figsize=(5, 5))
for i in range(1,N):
    A, B = linearize_dynamics(x_nom[i], u_nom[i], dt)
    C = np.array([[0.5*dt*np.cos(x_nom[i,2]), 0.5*dt*np.cos(x_nom[i,2])], 
                  [0.5*dt*np.sin(x_nom[i,2]), 0.5*dt*np.sin(x_nom[i,2])], 
                  [-dt/d, dt/d]])
    Sigma = A @ Sigma @ A.T + C @ Q @ C.T
    if i % 10 == 0:
        plot_ellipse(ax, x_nom[i], Sigma[:2,:2])

for i in range(N_MC):
    ax.plot(x_MC[i,0:N:10,0], x_MC[i,0:N:10,1], 'b.', markersize=1)
ax.axis('equal')

ax.set_xlim(-0.5, 0.5)
ax.set_ylim(0, 0.8)

Process uncertainty - noise in left and right wheel speeds

Measurement uncertainty - noise in ranging beacon measurements

In [ ]:
beacon_positions = np.array([[-1.0, -1.0], 
                             [5.0, -1.0], 
                             [-1.0, 5.0], 
                             [5.0, 5.0]])

def get_beacon_measurements(x, beacon_positions, sigma):
    """Return beacon measurements for a given state and beacon positions
    
    """
    z = np.zeros(beacon_positions.shape[0])
    for i in range(beacon_positions.shape[0]): 
        z[i] = np.linalg.norm(x[:2] - beacon_positions[i]) + np.random.normal(0, sigma)
    return z

def get_beacon_jacobian(x, beacon_positions):
    """Return Jacobian of beacon measurements for a given state and beacon positions
    
    """
    H = np.zeros((beacon_positions.shape[0], 3))
    for i in range(beacon_positions.shape[0]):
        H[i, :2] = (x[:2] - beacon_positions[i]) / np.linalg.norm(x[:2] - beacon_positions[i])
    return H

x = np.array([0.0, 0.0, 0.0])
sigma = 0.1
z = get_beacon_measurements(x, beacon_positions, sigma)

In [ ]:
def diff_drive_step(x, vl, vr, sigma, d, dt):
    """Diff drive model with uncertainty in the wheel velocities.

    d : distance between wheels

    """
    noise = np.random.normal(0, sigma, 2)
    dx = np.array([0.5 * (vl + vr + noise[0] + noise[1]) * np.cos(x[2]),
                    0.5 * (vl + vr + noise[0] + noise[1]) * np.sin(x[2]),
                    (vr - vl + noise[0] - noise[1]) / d])
    return x + dx * dt

In [ ]:
# Nominal trajectory
x0 = np.array([0.0, 0.0, 0.0])
dt = 0.1
d = 0.1
N = 300
vl = 0.195
vr = 0.2
v = (vl + vr) / 2
w = (vr - vl) / d
u_nom = np.array([[v, w]] * N)
x_nom = dubins_traj(x0, u_nom, dt)

plt.plot(x_nom[:, 0], x_nom[:, 1])
# Plot beacons
plt.plot(beacon_positions[:, 0], beacon_positions[:, 1], 'r*')
plt.axis('equal')
plt.show()

In [ ]:
from multirtd.dubins_reachability import dlqr_calculate


# Test EKF
x = np.array([0.0, 0.0, 0.0])
x_est = np.array([0.0, 0.0, 0.0])
P = 0.1 * np.eye(3)
motion_sigma = 0.01  # Uncertainty in left/right wheel velocities
range_sigma = 0.1
Q_motion = motion_sigma**2 * np.eye(2)
R_sense = range_sigma**2 * np.eye(4)

Q_lqr = np.eye(3)
R_lqr = np.eye(2)

x_traj = np.zeros((N, 3))
x_traj_est = np.zeros((N, 3))

for i in range(1, N):
    # Linearize about nominal trajectory
    A, B = linearize_dynamics(x_nom[i], u_nom[i], dt)
    C = np.array([[0.5*dt*np.cos(x_nom[i,2]), 0.5*dt*np.cos(x_nom[i,2])], 
                  [0.5*dt*np.sin(x_nom[i,2]), 0.5*dt*np.sin(x_nom[i,2])], 
                  [-dt/d, dt/d]])
    H = get_beacon_jacobian(x_nom[i], beacon_positions)

    # True dynamics
    K = dlqr_calculate(A, B, Q_lqr, R_lqr)
    u = u_nom[i] + K @ (x_nom[i] - x_est)
    vl = u[0] - u[1] * d / 2
    vr = u[0] + u[1] * d / 2
    x = diff_drive_step(x, vl, vr, motion_sigma, d, dt)

    # Measurement
    z = get_beacon_measurements(x, beacon_positions, range_sigma)
    
    # Predict step
    x_est = diff_drive_step(x_est, vl, vr, motion_sigma, d, dt)
    P = A @ P @ A.T + C @ Q_motion @ C.T
    # Update step
    # H = get_beacon_jacobian(x_est, beacon_positions)
    K = P @ H.T @ np.linalg.inv(H @ P @ H.T + R_sense)
    x_est = x_est + K @ (z - get_beacon_measurements(x_est, beacon_positions, range_sigma))
    P = (np.eye(P.shape[0]) - K @ H) @ P

    x_traj[i] = x
    x_traj_est[i] = x_est

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(x_traj[:, 0], x_traj[:, 1], 'b')
plt.plot(x_traj_est[:, 0], x_traj_est[:, 1], 'r')
plt.plot(x_nom[:, 0], x_nom[:, 1], 'g')
plt.plot(beacon_positions[:, 0], beacon_positions[:, 1], 'r*')
plt.axis('equal')
plt.show()

In [ ]:
def rollout(x0, u_nom, dt, N, beacon_positions, sigma):
    """Rollout for a given nominal trajectory
    
    """
    x = np.zeros((N, 3))
    x[0] = x0
    z = np.zeros((N, beacon_positions.shape[0]))
    for i in range(N-1):
        x[i+1] = diff_drive(x[i], u_nom[i], dt)
        z[i] = get_beacon_measurements(x[i], beacon_positions, sigma)
    return x, z

In [ ]:
from tqdm import tqdm

# Monte carlo rollouts

N_MC = 300


motion_sigma = 0.01 * np.ones(N)  # Uncertainty in left/right wheel velocities
# motion_sigma[100:200] = 0.05 
range_sigma = 0.01 * np.ones(N)
range_sigma[100:200] = 0.5 
# Q_motion = motion_sigma[0]**2 * np.eye(2)
# R_sense = range_sigma[0]**2 * np.eye(4)

Q_lqr = np.eye(3)
R_lqr = np.eye(2)

rollouts = np.zeros((N_MC, N, 3))

for k in tqdm(range(N_MC)):

    # Initialize
    x = np.array([0.0, 0.0, 0.0])
    x_est = np.array([0.0, 0.0, 0.0])
    P = 0.1 * np.eye(3)

    for i in range(1, N):
        Q_motion = motion_sigma[i]**2 * np.eye(2)
        R_sense = range_sigma[i]**2 * np.eye(4)
        
        # Linearize about nominal trajectory
        A, B = linearize_dynamics(x_nom[i], u_nom[i], dt)
        C = np.array([[0.5*dt*np.cos(x_nom[i,2]), 0.5*dt*np.cos(x_nom[i,2])], 
                    [0.5*dt*np.sin(x_nom[i,2]), 0.5*dt*np.sin(x_nom[i,2])], 
                    [-dt/d, dt/d]])
        H = get_beacon_jacobian(x_nom[i], beacon_positions)

        # True dynamics
        K = dlqr_calculate(A, B, Q_lqr, R_lqr)
        u = u_nom[i] + K @ (x_nom[i] - x_est)
        vl = u[0] - u[1] * d / 2
        vr = u[0] + u[1] * d / 2
        x = diff_drive_step(x, vl, vr, motion_sigma[i], d, dt)

        # Measurement
        z = get_beacon_measurements(x, beacon_positions, range_sigma[i])
        
        # Predict step
        x_est = diff_drive_step(x_est, vl, vr, motion_sigma[i], d, dt)
        P = A @ P @ A.T + C @ Q_motion @ C.T
        # Update step
        # H = get_beacon_jacobian(x_est, beacon_positions)
        K = P @ H.T @ np.linalg.inv(H @ P @ H.T + R_sense)
        x_est = x_est + K @ (z - get_beacon_measurements(x_est, beacon_positions, range_sigma[i]))
        P = (np.eye(P.shape[0]) - K @ H) @ P

        rollouts[k,i] = x

In [ ]:
plt.figure(figsize=(7, 7))
for i in range(N_MC):
    plt.plot(rollouts[i,:,0], rollouts[i,:,1], 'b', alpha=0.1)
plt.plot(x_nom[:, 0], x_nom[:, 1], 'g', linewidth=3)
plt.axis('equal')
plt.show()

RRBT Propagation

In [ ]:
Sigma = np.zeros((3, 3))  # state estimation covariance
Lambda = np.zeros((3, 3))  # uncertainty from not having yet taken observations

motion_sigma = 0.01 * np.ones(N)  # Uncertainty in left/right wheel velocities
# motion_sigma[100:200] = 0.05 
range_sigma = 0.01 * np.ones(N)
range_sigma[100:200] = 0.5 

fig, ax = plt.subplots(figsize=(5, 5))
for i in range(1,N):
    Q_motion = motion_sigma[i]**2 * np.eye(2)
    R_sense = range_sigma[i]**2 * np.eye(4)
    
    # Linearize about nominal trajectory
    A, B = linearize_dynamics(x_nom[i], u_nom[i], dt)
    C = np.array([[0.5*dt*np.cos(x_nom[i,2]), 0.5*dt*np.cos(x_nom[i,2])], 
                  [0.5*dt*np.sin(x_nom[i,2]), 0.5*dt*np.sin(x_nom[i,2])], 
                  [-dt/d, dt/d]])
    H = get_beacon_jacobian(x_nom[i], beacon_positions)
    K = dlqr_calculate(A, B, Q_lqr, R_lqr)
    
    # Covariance prediction
    Sigma = A @ Sigma @ A.T + C @ Q_motion @ C.T
    L = Sigma @ H.T @ np.linalg.inv(H @ Sigma @ H.T + R_sense)

    # Covariance update
    Lambda = (A - B @ K) @ Lambda @ (A - B @ K).T + L @ H @ Sigma
    Sigma = Sigma - L @ H @ Sigma

    if i % 10 == 0:
        plot_ellipse(ax, x_nom[i], Sigma[:2,:2] + Lambda[:2,:2])

# for i in range(N_MC):
#     ax.plot(rollouts[i,0:N:10,0], rollouts[i,0:N:10,1], 'b.', markersize=1)
ax.axis('equal')

# ax.set_xlim(-0.5, 0.5)
# ax.set_ylim(0, 0.8)

In [ ]:
Lambda

In [ ]:
B @ K

In [ ]:
K